<a href="https://colab.research.google.com/github/gylam/siads696-sprsum2024-team05/blob/main/RP_Preprocessing_code_only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#stopwords
def create_stopwords():
    """creating a comprehensive list with nltk stopwords, json-en, punctuation"""
    #nltk stopwords
    ext_stopwords = stopwords.words('english')

    #json-en stopwords
    json_stop = requests.get("https://raw.githubusercontent.com/6/stopwords-json/master/dist/en.json").json()
    ext_stopwords =  set(ext_stopwords + json_stop + punctuation.split())
    return ext_stopwords

#text preprocessing code - the main function is pos_lemmatizer that call support functions remove_urls, tokenizer and penn2morphy
def remove_urls(text):
    """remove urls from the data"""
    url_pattern = r'https?://\S+|www\.\S+'
    text_without_urls = re.sub(url_pattern, '', text)
    return text_without_urls

def tokenizer(text):
    """tokenize the extracted text and remove stopwords/punctuation etc."""
    return [token for token in nltk.word_tokenize(text) if token.lower() not in ext_stopwords and len(token) >2
            and re.search('^[a-zA-Z]{3,}$', token)]

def penn2morphy(penntag):
    """Converts Penn Treebank tags to WordNet per here: https://www.kaggle.com/code/alvations/basic-nlp-with-nltk"""
    morphy_tag = {'NN':'n', 'JJ':'a', 'VB':'v', 'RB':'r'}
    try: return morphy_tag[penntag[:2]]
    except:return 'n' # if mapping isn't found, fall back to Noun.

def pos_lemmatizer(text):
    """pos tagging and lemmatization"""

    pos_tokens = nltk.pos_tag(tokenizer(remove_urls(text)))
    # useful explanation of what the tags mean: https://stackoverflow.com/questions/15388831/what-are-all-possible-pos-tags-of-nltk
    wnl = WordNetLemmatizer()
    wnl_tokens = []
    for token, tag in pos_tokens:
        if tag in ["NNP", "NNPS", "CD"]: #Dropping proper nouns and numericals (CD)
            pass
        else:
            wnl_tokens.append(wnl.lemmatize(token, pos = penn2morphy(tag)).lower()) #these are the tokens we want to use in the vectorizer

    return " ".join(wnl_tokens)

ext_stopwords = create_stopwords()

t0 = time.time()
df["cleaned_text"] = df["combined_text"].apply(pos_lemmatizer)
t1 = time.time()
print(f"time taken to preprocess data:{round((t1-t0)/60,2)}")
# df.to_pickle("dataset_clean.pkl")
# df = pd.read_pickle("/home/prabhur/dataset_clean.pkl")
print(df.shape)
df.head()